In [ ]:
# import pandas as pd
# import numpy as np

# # Read the CSV files
# csv1 = "/content/Queries_Dataset.csv"  # Replace with the path to your first CSV file
# csv2 = "/content/selected_questions.csv"  # Replace with the path to your second CSV file

# data1 = pd.read_csv(csv1)

# print(len(data1))
# data2 = pd.read_csv(csv2)

# # Rename column 'Target' to 'target' in the second dataframe
# if 'Target' in data2.columns:
#     data2.rename(columns={'Target': 'target'}, inplace=True)

# # Append the dataframes
# data_combined = pd.concat([data1, data2], ignore_index=True)

# # Shuffle the rows randomly
# data_shuffled = data_combined.sample(frac=1, random_state=42).reset_index(drop=True)

# # Save the shuffled data to a new CSV
# output_csv = "shuffled_output.csv"  # Replace with the desired output file name
# data_shuffled.to_csv(output_csv, index=False)

# print(f"Shuffled data saved to {output_csv}")

2000
Shuffled data saved to shuffled_output.csv


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import joblib  # Import joblib for saving models


In [ ]:

# Step 1: Load the dataset
data = pd.read_csv('/content/shuffled_output.csv')

# Step 2: Split the dataset into train, test, and validation sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)  # 80% train, 20% test
train_data, val_data = train_test_split(train_data, test_size=0.1, random_state=42)  # Split train into 90% train, 10% validation

# Step 3: Prepare the data using TF-IDF vectorization
tfidf = TfidfVectorizer(max_features=5000)  # Limit to top 5000 features

# Fit and transform on training data, then transform on validation and test data
X_train_tfidf = tfidf.fit_transform(train_data['Question'])
X_val_tfidf = tfidf.transform(val_data['Question'])
X_test_tfidf = tfidf.transform(test_data['Question'])

y_train = train_data['target']
y_val = val_data['target']
y_test = test_data['target']


In [ ]:
logreg = LogisticRegression(max_iter=1000, C=0.1)  # Adjust C for regularization

svm = LinearSVC()
nb = MultinomialNB()

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import log_loss, accuracy_score
import numpy as np

# Helper function to track metrics at each iteration
def track_metrics(model, X_train, y_train, X_val, y_val, epochs):
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []

    for epoch in range(epochs):
        model.partial_fit(X_train, y_train, classes=np.unique(y_train))  # Update model

        # Train predictions
        train_preds = model.predict(X_train)
        train_probs = model.predict_proba(X_train)
        train_loss = log_loss(y_train, train_probs)
        train_acc = accuracy_score(y_train, train_preds)

        # Validation predictions
        val_preds = model.predict(X_val)
        val_probs = model.predict_proba(X_val)
        val_loss = log_loss(y_val, val_probs)
        val_acc = accuracy_score(y_val, val_preds)

        # Track losses and accuracies
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accuracies.append(train_acc)
        val_accuracies.append(val_acc)

        print(f"Epoch {epoch + 1}/{epochs}")
        print(f"Train Loss: {train_loss}, Validation Loss: {val_loss}")
        print(f"Train Accuracy: {train_acc}, Validation Accuracy: {val_acc}")
        print("-" * 50)

    return train_losses, val_losses, train_accuracies, val_accuracies


In [ ]:

# Logistic Regression via SGDClassifier
logreg_sgd = SGDClassifier(loss='log_loss', max_iter=1, warm_start=True)  # Logistic regression
train_losses_logreg, val_losses_logreg, train_acc_logreg, val_acc_logreg = track_metrics(
    logreg_sgd, X_train_tfidf, y_train, X_val_tfidf, y_val, epochs=10
)



Epoch 1/10
Train Loss: 0.05065367225280855, Validation Loss: 0.05478695377967704
Train Accuracy: 0.999537037037037, Validation Accuracy: 1.0
--------------------------------------------------
Epoch 2/10
Train Loss: 0.05290854971768224, Validation Loss: 0.0564500550129149
Train Accuracy: 0.9990740740740741, Validation Accuracy: 1.0
--------------------------------------------------
Epoch 3/10
Train Loss: 0.05449088559330984, Validation Loss: 0.058777773766297114
Train Accuracy: 0.9990740740740741, Validation Accuracy: 1.0
--------------------------------------------------
Epoch 4/10
Train Loss: 0.0536714349321995, Validation Loss: 0.05835989179293263
Train Accuracy: 0.9990740740740741, Validation Accuracy: 1.0
--------------------------------------------------
Epoch 5/10
Train Loss: 0.053941735930894776, Validation Loss: 0.05780949421076968
Train Accuracy: 0.9990740740740741, Validation Accuracy: 1.0
--------------------------------------------------
Epoch 6/10
Train Loss: 0.05494656495

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.utils import shuffle
from sklearn.metrics import hinge_loss, accuracy_score
import numpy as np

# Adjust learning rate if needed by tuning `eta0`
svm_sgd = SGDClassifier(loss='hinge', max_iter=1, eta0=0.01, learning_rate='constant')

def track_metrics_svm(model, X_train, y_train, X_val, y_val, epochs):
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []

    for epoch in range(epochs):
        # Shuffle the data at the beginning of each epoch
        X_train, y_train = shuffle(X_train, y_train, random_state=epoch)

        model.partial_fit(X_train, y_train, classes=np.unique(y_train))  # Update model

        # Train predictions
        train_preds = model.decision_function(X_train)
        train_loss = hinge_loss(y_train, train_preds)
        train_acc = accuracy_score(y_train, model.predict(X_train))

        # Validation predictions
        val_preds = model.decision_function(X_val)
        val_loss = hinge_loss(y_val, val_preds)
        val_acc = accuracy_score(y_val, model.predict(X_val))

        # Track losses and accuracies
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accuracies.append(train_acc)
        val_accuracies.append(val_acc)

        print(f"Epoch {epoch + 1}/{epochs}")
        print(f"SVM Train Loss (Hinge): {train_loss}, Validation Loss (Hinge): {val_loss}")
        print(f"SVM Train Accuracy: {train_acc}, Validation Accuracy: {val_acc}")
        print("-" * 50)

    return train_losses, val_losses, train_accuracies, val_accuracies

# Assuming X_train_tfidf, y_train, X_val_tfidf, and y_val are already defined
train_losses_svm, val_losses_svm, train_acc_svm, val_acc_svm = track_metrics_svm(
    svm_sgd, X_train_tfidf, y_train, X_val_tfidf, y_val, epochs=10
)


Epoch 1/10
SVM Train Loss (Hinge): 0.40909811606026003, Validation Loss (Hinge): 0.40702759352472345
SVM Train Accuracy: 0.8912037037037037, Validation Accuracy: 0.8625
--------------------------------------------------
Epoch 2/10
SVM Train Loss (Hinge): 0.26256073642890193, Validation Loss (Hinge): 0.2867128217683144
SVM Train Accuracy: 0.9222222222222223, Validation Accuracy: 0.8833333333333333
--------------------------------------------------
Epoch 3/10
SVM Train Loss (Hinge): 0.19259759796620904, Validation Loss (Hinge): 0.21510329414716445
SVM Train Accuracy: 0.9532407407407407, Validation Accuracy: 0.925
--------------------------------------------------
Epoch 4/10
SVM Train Loss (Hinge): 0.1388173309640285, Validation Loss (Hinge): 0.1619571484512893
SVM Train Accuracy: 0.9777777777777777, Validation Accuracy: 0.9625
--------------------------------------------------
Epoch 5/10
SVM Train Loss (Hinge): 0.09760276659708061, Validation Loss (Hinge): 0.11688661973648319
SVM Train A

In [ ]:

# Naive Bayes (MultinomialNB) training with tracking
nb = MultinomialNB()
train_losses_nb, val_losses_nb, train_acc_nb, val_acc_nb = track_metrics(
    nb, X_train_tfidf, y_train, X_val_tfidf, y_val, epochs=10
)

Epoch 1/10
Train Loss: 0.10642100160927634, Validation Loss: 0.1317635342352663
Train Accuracy: 0.9986111111111111, Validation Accuracy: 0.9666666666666667
--------------------------------------------------
Epoch 2/10
Train Loss: 0.05677904513877317, Validation Loss: 0.09125140881540651
Train Accuracy: 0.999537037037037, Validation Accuracy: 0.9666666666666667
--------------------------------------------------
Epoch 3/10
Train Loss: 0.03761897181243509, Validation Loss: 0.0747892689389664
Train Accuracy: 0.999537037037037, Validation Accuracy: 0.9666666666666667
--------------------------------------------------
Epoch 4/10
Train Loss: 0.02769490683306599, Validation Loss: 0.06571518935381784
Train Accuracy: 1.0, Validation Accuracy: 0.9666666666666667
--------------------------------------------------
Epoch 5/10
Train Loss: 0.02173285439290335, Validation Loss: 0.05992358758463288
Train Accuracy: 1.0, Validation Accuracy: 0.9708333333333333
---------------------------------------------

In [ ]:

# Step 4: Train and evaluate models

# Logistic Regression

logreg.fit(X_train_tfidf, y_train)
y_val_pred_logreg = logreg.predict(X_val_tfidf)
print(f"Logistic Regression Validation Accuracy: {accuracy_score(y_val, y_val_pred_logreg)}")

# SVM (LinearSVC)

svm.fit(X_train_tfidf, y_train)
y_val_pred_svm = svm.predict(X_val_tfidf)
print(f"SVM Validation Accuracy: {accuracy_score(y_val, y_val_pred_svm)}")

# Naive Bayes (MultinomialNB)

nb.fit(X_train_tfidf, y_train)
y_val_pred_nb = nb.predict(X_val_tfidf)
print(f"Naive Bayes Validation Accuracy: {accuracy_score(y_val, y_val_pred_nb)}")



Logistic Regression Validation Accuracy: 0.9875
SVM Validation Accuracy: 1.0
Naive Bayes Validation Accuracy: 0.9666666666666667


In [ ]:

# Step 5: Test the models on the test set
print("\nTest Set Performance:")
y_test_pred_logreg = logreg.predict(X_test_tfidf)
print(f"Logistic Regression Test Accuracy: {accuracy_score(y_test, y_test_pred_logreg)}")

y_test_pred_svm = svm.predict(X_test_tfidf)
print(f"SVM Test Accuracy: {accuracy_score(y_test, y_test_pred_svm)}")

y_test_pred_nb = nb.predict(X_test_tfidf)
print(f"Naive Bayes Test Accuracy: {accuracy_score(y_test, y_test_pred_nb)}")

# Step 6: Evaluate the Logistic Regression model (or you can choose another)
print("\nLogistic Regression Evaluation Metrics:")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_test_pred_logreg))
print("\nClassification Report:\n", classification_report(y_test, y_test_pred_logreg))

print("\nSVM Evaluation Metrics:")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_test_pred_svm))
print("\nClassification Report:\n", classification_report(y_test, y_test_pred_svm))

print("\nNiave bayes Evaluation Metrics:")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_test_pred_nb))
print("\nClassification Report:\n", classification_report(y_test, y_test_pred_nb))

# Step 7: Save the trained models
joblib.dump(logreg, '/content/logistic_regression_model.pkl')
joblib.dump(svm, '/content/svm_model.pkl')
joblib.dump(nb, '/content/naive_bayes_model.pkl')
joblib.dump(tfidf, '/content/tfidf_vectorizer.pkl')  # Save the vectorizer as well

print("\nModels saved successfully!")


Test Set Performance:
Logistic Regression Test Accuracy: 0.9866666666666667
SVM Test Accuracy: 0.9933333333333333
Naive Bayes Test Accuracy: 0.985

Logistic Regression Evaluation Metrics:
Confusion Matrix:
 [[195   1   0]
 [  0 212   7]
 [  0   0 185]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.99      1.00       196
           1       1.00      0.97      0.98       219
           2       0.96      1.00      0.98       185

    accuracy                           0.99       600
   macro avg       0.99      0.99      0.99       600
weighted avg       0.99      0.99      0.99       600


SVM Evaluation Metrics:
Confusion Matrix:
 [[195   1   0]
 [  0 216   3]
 [  0   0 185]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.99      1.00       196
           1       1.00      0.99      0.99       219
           2       0.98      1.00      0.99       185

    acc

In [ ]:
import joblib
import pandas as pd

# Load the saved models and vectorizer
logreg_model = joblib.load('/content/logistic_regression_model.pkl')
svm_model = joblib.load('/content/svm_model.pkl')
nb_model = joblib.load('/content/naive_bayes_model.pkl')
tfidf_vectorizer = joblib.load('/content/tfidf_vectorizer.pkl')

# Function to make predictions
def predict_query_type(question):
    # Transform the input question using the TF-IDF vectorizer
    question_tfidf = tfidf_vectorizer.transform([question])

    # Make predictions with each model
    logreg_prediction = logreg_model.predict(question_tfidf)[0]
    svm_prediction = svm_model.predict(question_tfidf)[0]
    nb_prediction = nb_model.predict(question_tfidf)[0]

    # Return the predictions
    return {
        'Logistic Regression Prediction': logreg_prediction,
        'SVM Prediction': svm_prediction,
        'Naive Bayes Prediction': nb_prediction
    }

# Example question
question = 'What is status of pie driver in stock?'
predictions = predict_query_type(question)

# Print the predictions
print(predictions)


{'Logistic Regression Prediction': 2, 'SVM Prediction': 2, 'Naive Bayes Prediction': 2}
